In [8]:
#importing in necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [27]:
#reading in the prepared training data.  currently, data is not scaled.
df = pd.read_excel('data/binned2.xlsx')
df.head()

,age,sex,cp,trestbps,chol,fbs,thalrest,exang,num
0,63,1,1,145,233,1,60,0,0
1,67,1,4,160,286,0,64,1,1
2,67,1,4,120,229,0,78,1,1
3,37,1,3,130,250,0,84,0,0
4,41,0,2,130,204,0,71,0,0


In [28]:
#need to define the attributes and labels (aka, CAD diagnosis)
y = df['num']
X = df.drop(['num'], axis=1)

In [29]:
#defining the training and test sets; going with 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [30]:
#training the Random Forest model
classifier = RandomForestClassifier(n_estimators=800, bootstrap=False, random_state=0)  
classifier.fit(X_train, y_train)  
y_pred = classifier.predict(X_test) 

In [31]:
#evaluating the model
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[65 19]
 [18 78]]
              precision    recall  f1-score   support

           0       0.78      0.77      0.78        84
           1       0.80      0.81      0.81        96

   micro avg       0.79      0.79      0.79       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.79      0.79      0.79       180

0.7944444444444444


## Need to bin the chest pain data to match NHANES data and rerun the model.  then run additional performance metrics, print a decision tree to see how complicated it is, cross-validate, can do a  RandomizedSearchCV to try to improve model parameters.  Finally, pickle it.